In [4]:
import pandas as pd
import sqlite3

In [5]:
connect = sqlite3.connect('../../datasets/checking-logs.sqlite')

create a new table by joining the tables pageviews and checker

In [6]:
connect.execute(""" CREATE TABLE datamart AS
SELECT c.uid, c.labname, c.timestamp AS first_commit_ts, MIN(p.datetime) as first_view_ts FROM checker c
LEFT JOIN pageviews p ON c.uid = p.uid
WHERE c.status = 'ready' AND c.numTrials = 1 AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1') AND c.uid LIKE 'user_%'
GROUP BY c.uid, c.labname, c.timestamp """)
connect.commit()

OperationalError: table datamart already exists

In [7]:
datamart = pd.read_sql('SELECT * FROM datamart', connect, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart['first_commit_ts'] = pd.to_datetime(datamart['first_commit_ts'])
datamart['first_view_ts'] = pd.to_datetime(datamart['first_view_ts'])

In [8]:
datamart.info()
datamart

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,NaT
136,user_8,laba05,2020-05-02 13:28:07.705193,NaT
137,user_8,laba06,2020-05-16 17:56:15.755553,NaT
138,user_8,laba06s,2020-05-16 20:01:07.900727,NaT


Create two dataframes and save both tables into the database

In [9]:
test = datamart[datamart['first_view_ts'].notna()]
control = datamart[datamart['first_view_ts'].isna()]
control = control.fillna(test['first_view_ts'].mean())


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [11]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [12]:
control.to_sql('control',connect,if_exists='replace')
test.to_sql('test',connect,if_exists='replace')

59

In [13]:
connect.close()